In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats as stats
import pandas as pd  
from pandas import Series, DataFrame

Item-Based

In [49]:
#前半部分代码同基于用户的协同过滤

#基于用户的协同过滤
critics={\
'Lisa Rose':{'lady in water':2.5,'snakes on a plane':3.5,'just my luck':3.0,'superman returns':3.5,'you,me and dupree':2.5,'the night listener':3.0},

'Gene Seymour':{'lady in water':3.0,'snakes on a plane':3.5,'just my luck':1.5,'superman returns':5.0,'you,me and dupree':3.5,'the night listener':3.0},

'Michael Phillips':{'lady in water':2.5,'snakes on a plane':3.0,'superman returns':3.5,'the night listener':4.0},

'Claudia Puig':{'snakes on a plane':3.5,'just my luck':3.0,'superman returns':4.0,'the night listener':4.5},

'Mick Lasalle':{'lady in water':3.0,'snakes on a plane':4.0,'just my luck':2.0,'superman returns':3.0,'you,me and dupree':2.0,'the night listener':3.0},

'Jack Matthews':{'lady in water':3.0,'snakes on a plane':4.0,'superman returns':5.0,'you,me and dupree':3.5,'the night listener':3.0},

'Toby':{'snakes on a plane':4.5,'superman returns':4.0,'you,me and dupree':1.0}}

from math import sqrt
#基于用户的协同过滤
#欧几里德距离
def sim_distance(prefs, p1, p2):
    si = {}
    # find common items
    for item in prefs[p1]:
        if item in prefs[p2]:  si[item] = 1
    if len(si) == 0:  return 0
    #cal the distance
    sum_of_sqr = sum([pow(prefs[p1][item] - prefs[p2][item], 2) for item in si]) 
    return 1 / (1 + sqrt(sum_of_sqr))

#基于用户的协同过滤
#皮尔逊相关度
def sim_pearson(prefs, p1, p2):
    si = {}
    for item in prefs[p1]:
        if item in prefs[p2]: si[item] = 1
    if len(si) == 0: return 0
    # 对所有偏好求和
    sum1 = sum([prefs[p1][item] for item in si])
    sum2 = sum([prefs[p2][item] for item in si])
    # 求平方和
    sum1sq = sum([pow(prefs[p1][item], 2) for item in si])
    sum2sq = sum([pow(prefs[p2][item], 2) for item in si])
    # 求乘积之和
    psum = sum([prefs[p1][item] * prefs[p2][item] for item in si])
    # 计算皮尔逊相关度
    num = psum - sum1 * sum2 / len(si)
    den = sqrt((sum1sq - pow(sum1, 2) / len(si)) * (sum2sq - pow(sum2, 2) / len(si)))
    if den == 0: return 0
    return num / den

#基于用户——>基于物品
#可以代码复用，只需要将人和物品对调即可。
#物品与人员互相调换
def transformprefs(prefs):
    result = {}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item, {})
            # 人、物对调
            result[item][person] = prefs[person][item]
    return result

#最相关的N个person，这里仅仅是函数定义，实际传入参数为物品
def topmatchs(prefs, person, n=5, similarity=sim_pearson):
    scores = [(similarity(prefs, person, other), other) for other in prefs if other != person]
    scores.sort(reverse=True)
    return scores[0:n]

#构建物品比较数据集合,即，每个item有n个最相关的其它item
def calculatesimilaryitems(prefs, n=10):
    itemsim = {}
    itemprefs = transformprefs(prefs)
    c = 0
    for item in itemprefs:
        c += 1.0
        if c%100 == 0: print('%d/%d' %(c, len(itemprefs)))
        scores = topmatchs(itemprefs, item, n=n, similarity=sim_pearson)
        itemsim[item] = scores
    return itemsim

#来了个新用户，获得推荐
def getrecommendations(prefs, itemsim, user):
    totalsim = {}
    simsum = {}
    for item, ratio in prefs[user].items():
        for similarity, item2 in itemsim[item]:
            if item2 in prefs[user]: continue
            totalsim.setdefault(item2, 0)
            totalsim[item2] += similarity*ratio
            simsum.setdefault(item2, 0)
            simsum[item2] += similarity
    rankings = [(total/simsum[item], item) for item, total in totalsim.items()]
    rankings.sort(reverse=True)
    return rankings

print(getrecommendations(critics, calculatesimilaryitems(critics), 'Toby'))

[(4.379491181049615, 'the night listener'), (3.610031066802182, 'lady in water'), (2.938617629204515, 'just my luck')]


In [50]:
def load_data(path):
    data = []
    '''
    f = open(filename, 'rb').read()
    data = f.decode().split('\n')
    '''
    with open(path, 'rb') as file:
        for line in file:
            (user_id, moive_id, rating, time_stamp) = line.strip().split('\t'.encode(encoding="utf-8"))
            data.append([user_id, moive_id, rating])
    data = np.array(data).astype(np.uint16)
    return data

class  item_based_cf:
    def __init__(self, train_data):
        self.train_data = np.array(train_data)
        print("the input data size is ", self.train_data.shape)
        self.movie_user = {}
        self.user_movie = {}
        self.avg = np.mean(self.train_data[:, 2])  # 不知道干啥的时候用
        for i in range(self.train_data.shape[0]):
            # 逐行，读取数据
            user_id = self.train_data[i][0]
            moive_id = self.train_data[i][1]
            rating = self.train_data[i][2]
            # 把需要考虑的情况全部添加键值对
            # setdefualt添加键值对，下面内部继续赋值
            self.movie_user.setdefault(moive_id, {})
            self.user_movie.setdefault(user_id, {})
            self.movie_user[moive_id][user_id] = rating
            self.user_movie[user_id][moive_id] = rating
        self.similarity = {}

    def sim_cal(self, m1, m2):
        # 比较两者相似性
        self.similarity.setdefault(m1, {})
        self.similarity[m1][m2] = -1     #等价于self.similarity[m1].setdefault(m2,-1)  两层字典{m1:{m2:-1}}
        self.similarity.setdefault(m2, {})
        self.similarity[m2][m1] = -1

        # movie_user中间。m1, m2类别初始化，已经有了的类别不会改变原来的键值对
        self.movie_user.setdefault(m1, {})
        self.movie_user.setdefault(m2, {})

        '''
        老用户直接返回
        '''
        if self.similarity[m1][m2] != -1:
            return self.similarity[m1][m2]

        # 都评价过m1, m2的用户。格式 {userid：1}
        su = {}
        for user in self.movie_user[m1]:
            if user in self.movie_user[m2]:
                su[user] = 1

        # 没有交集，返回 1
        n = len(su)
        if (n == 0):
            self.similarity[m1][m2] = 1
            self.similarity[m2][m1] = 1
            return 1

        s1 = np.array([self.movie_user[m1][u] for u in su])  #返回一大堆rating
        s2 = np.array([self.movie_user[m2][u] for u in su])
        # 求和
        sum1 = np.sum(s1)
        sum2 = np.sum(s2)
        #求平方和
        sum1Sq = np.sum(s1**2)
        sum2Sq = np.sum(s2**2)

        #计算皮尔逊相关系数
        pSum = np.sum(s1*s2)  # 内积
        num = pSum-(sum1*sum2/n)
        den = np.sqrt((sum1Sq-sum1**2/n)*(sum2Sq-sum2**2/n))  # 两者方差积——分母

        if den == 0:  # 一般不可能发生
            self.similarity[m1][m2] = 0
            self.similarity[m2][m1] = 0
            return 0

        self.similarity[m1][m2] = num/den
        self.similarity[m2][m1] = num/den
        return num/den

    '''
    Predict whether a user like this item, I think this way is too sample. But it's a way.
    '''
    def predict(self, user_id, moive_id):
        '''initialization'''
        sim_accumulate = 0.0
        rat_acc = 0.0
        for item in self.user_movie[user_id]:
            # 用户喜欢物品的逐个物体运算
            sim = self.sim_cal(item, moive_id)
            if sim < 0:
                continue
            '''renew'''
            rat_acc += sim * self.user_movie[user_id][item]  # 关系大小和原来物体乘积
            sim_accumulate += sim
        '''no same user rated,return average rates of the data'''
        if sim_accumulate == 0:
            return self.avg
        return rat_acc/sim_accumulate  # At last，likehood average rates

    # 修改文件格式，把结果整理形式
    def test(self, test_X):
        test_X = np.array(test_X)
        output = []
        sums = 0
        print("the test data size is ", test_X.shape)
        for i in range(test_X.shape[0]):
            pre = self.predict(test_X[i][0], test_X[i][1])
            output.append(pre)
            sums += (pre-test_X[i][2])**2
        rmse = np.sqrt(sums/test_X.shape[0])
        print("the rmse on test data is ", rmse)
        return output

train_data = load_data('./ml-100k/u1.base')
test_data = load_data('./ml-100k/u1.test')

ibc = item_based_cf(train_data)
pred = ibc.test(test_data)

the input data size is  (80000, 3)
the test data size is  (20000, 3)
the rmse on test data is  1.0416491735485909


User-Based

In [38]:
from math import sqrt
 
dataset = {
			'Lisa Rose': {'Lady in the Water': 2.5, 
							'Snakes on a Plane': 3.5,
							'Just My Luck': 3.0, 
							'Superman Returns': 3.5, 
							'You, Me and Dupree': 2.5,
							'The Night Listener': 3.0},
			'Gene Seymour': {'Lady in the Water': 3.0, 
							'Snakes on a Plane': 3.5,
							'Just My Luck': 1.5,
							'Superman Returns': 5.0, 
							'The Night Listener': 3.0,
							'You, Me and Dupree': 3.5},
			'Michael Phillips': {'Lady in the Water': 2.5, 
								'Snakes on a Plane': 3.0,
								'Superman Returns': 3.5,
								 'The Night Listener': 4.0},
			'Claudia Puig': {'Snakes on a Plane': 3.5, 
							'Just My Luck': 3.0,
							'The Night Listener': 4.5, 
							'Superman Returns': 4.0,
							'You, Me and Dupree': 2.5},
			'Mick LaSalle': {'Lady in the Water': 3.0, 
							'Snakes on a Plane': 4.0,
							'Just My Luck': 2.0, 
							'Superman Returns': 3.0, 
							'The Night Listener': 3.0,
							'You, Me and Dupree': 2.0},
			'Jack Matthews': {'Lady in the Water': 3.0, 
							'Snakes on a Plane': 4.0,
							'The Night Listener': 3.0, 
							'Superman Returns': 5.0, 
							'You, Me and Dupree': 3.5},
			'Toby': {'Snakes on a Plane':4.5,
					'You, Me and Dupree':1.0,
					'Superman Returns':4.0}}

def similarity_score(person1, person2):
	# Returns ratio Euclidean distance score of person1 and person2 
 
	both_viewed = {} # To get both rated items by person1 and person2
 
	for item in dataset[person1]:
		if item in dataset[person2]:
			both_viewed[item] = 1
 
		# Conditions to check they both have an common rating items	
		if len(both_viewed) == 0:
			return 0
 
		# Finding Euclidean distance 
		sum_of_eclidean_distance = []
 
		for item in dataset[person1]:
			if item in dataset[person2]:
				sum_of_eclidean_distance.append(pow(dataset[person1][item] - dataset[person2][item], 2))
		sum_of_eclidean_distance = sum(sum_of_eclidean_distance)
 
		return 1/(1+sqrt(sum_of_eclidean_distance))

def pearson_correlation(person1, person2):
	# To get both rated items
	both_rated = {}
	for item in dataset[person1]:
		if item in dataset[person2]:
			both_rated[item] = 1
 
	number_of_ratings = len(both_rated)

	# Checking for number of ratings in common
	if number_of_ratings == 0:
		return 0
 
	# Add up all the preferences of each user
	person1_preferences_sum = sum([dataset[person1][item] for item in both_rated])
	person2_preferences_sum = sum([dataset[person2][item] for item in both_rated])
 
	# Sum up the squares of preferences of each user
	person1_square_preferences_sum = sum([pow(dataset[person1][item], 2) for item in both_rated])
	person2_square_preferences_sum = sum([pow(dataset[person2][item], 2) for item in both_rated])
 
	# Sum up the product value of both preferences for each item
	product_sum_of_both_users = sum([dataset[person1][item] * dataset[person2][item] for item in both_rated])
 
	# Calculate the pearson score
	numerator_value = product_sum_of_both_users - (person1_preferences_sum*person2_preferences_sum/number_of_ratings)
	denominator_value = sqrt((person1_square_preferences_sum - pow(person1_preferences_sum,2)/number_of_ratings) * (person2_square_preferences_sum -pow(person2_preferences_sum, 2)/number_of_ratings))
	if denominator_value == 0:
		return 0
	else:
		r = numerator_value/denominator_value
		return r 

def most_similar_users(person, number_of_users):
	# returns the number_of_users (similar persons) for a given specific person.
	scores = [(pearson_correlation(person, other_person), other_person) for other_person in dataset if other_person != person ]

	# Sort the similar persons so that highest scores person will appear at the first
	scores.sort()
	scores.reverse()
	return scores[0:number_of_users]

def user_reommendations(person):
	# Gets recommendations for a person by using a weighted average of every other user's rankings
	totals = {}
	simSums = {}
	rankings_list = []
	for other in dataset:
		# don't compare me to myself
		if other == person:
			continue
		sim = pearson_correlation(person, other)
		print(other + ">>>>>>>", sim)
 
		# ignore scores of zero or lower
		if sim <= 0: 
			continue
		for item in dataset[other]:
			# only score movies i haven't seen yet
			if item not in dataset[person] or dataset[person][item] == 0:
			# Similrity * score
				totals.setdefault(item, 0)
				totals[item] += dataset[other][item] * sim
				# sum of similarities
				simSums.setdefault(item, 0)
				simSums[item] += sim
 
	# Create the normalized list
	rankings = [(total/simSums[item], item) for item, total in totals.items()]
	rankings.sort()
	rankings.reverse()
	# returns the recommended items
	recommendataions_list = [(recommend_item, score) for score, recommend_item in rankings]
	return recommendataions_list

print(user_reommendations('Toby'))

Lisa Rose>>>>>>> 0.9912407071619299
Gene Seymour>>>>>>> 0.38124642583151164
Michael Phillips>>>>>>> -1.0
Claudia Puig>>>>>>> 0.8934051474415647
Mick LaSalle>>>>>>> 0.9244734516419049
Jack Matthews>>>>>>> 0.66284898035987
[('The Night Listener', 3.3477895267131017), ('Lady in the Water', 2.8325499182641614), ('Just My Luck', 2.530980703765565)]


In [46]:
#基于用户的协同过滤
critics={\
'Lisa Rose':{'lady in water':2.5,'snakes on a plane':3.5,'just my luck':3.0,'superman returns':3.5,'you,me and dupree':2.5,'the night listener':3.0},

'Gene Seymour':{'lady in water':3.0,'snakes on a plane':3.5,'just my luck':1.5,'superman returns':5.0,'you,me and dupree':3.5,'the night listener':3.0},

'Michael Phillips':{'lady in water':2.5,'snakes on a plane':3.0,'superman returns':3.5,'the night listener':4.0},

'Claudia Puig':{'snakes on a plane':3.5,'just my luck':3.0,'superman returns':4.0,'the night listener':4.5},

'Mick Lasalle':{'lady in water':3.0,'snakes on a plane':4.0,'just my luck':2.0,'superman returns':3.0,'you,me and dupree':2.0,'the night listener':3.0},

'Jack Matthews':{'lady in water':3.0,'snakes on a plane':4.0,'superman returns':5.0,'you,me and dupree':3.5,'the night listener':3.0},

'Toby':{'snakes on a plane':4.5,'superman returns':4.0,'you,me and dupree':1.0}}

from math import sqrt
#欧几里德距离
def sim_distance(prefs, p1, p2):
    si = {}
    # find common items
    for item in prefs[p1]:
        if item in prefs[p2]:  si[item] = 1
    if len(si) == 0:  return 0
    #cal the distance
    sum_of_sqr = sum([pow(prefs[p1][item] - prefs[p2][item], 2) for item in si]) 
    return 1 / (1 + sqrt(sum_of_sqr))

#皮尔逊相关度
def sim_pearson(prefs, p1, p2):
    si = {}
    for item in prefs[p1]:
        if item in prefs[p2]: si[item] = 1
    if len(si) == 0: return 0
    # 对所有偏好求和
    sum1 = sum([prefs[p1][item] for item in si])
    sum2 = sum([prefs[p2][item] for item in si])
    # 求平方和
    sum1sq = sum([pow(prefs[p1][item], 2) for item in si])
    sum2sq = sum([pow(prefs[p2][item], 2) for item in si])
    # 求乘积之和
    psum = sum([prefs[p1][item] * prefs[p2][item] for item in si])
    # 计算皮尔逊相关度
    num = psum - sum1 * sum2 / len(si)
    den = sqrt((sum1sq - pow(sum1, 2) / len(si)) * (sum2sq - pow(sum2, 2) / len(si)))
    if den == 0: return 0
    return num / den

#获得推荐
def get_recommendations(prefs, person, similarity):
    totals = {}
    simsum = {}
    for other in prefs:
        #dont compare to myself
        if other == person: continue
        sim = similarity(prefs, person, other)
        #loop out if there is no similarity
        if sim <= 0 : continue
        for item in prefs[other]:
            #recommend a film I have never watched
            if item not in prefs[person]:
                #value*sim
                totals.setdefault(item, 0)
                totals[item] += prefs[other][item] * sim
                #sum of sim
                simsum.setdefault(item, 0)
                simsum[item] += sim
    #recommand list 
    rankings = [(value/simsum[item], item) for item, value in totals.items()] #recommand list 
    rankings.sort(reverse=True)
    return rankings

print(get_recommendations(critics, 'Toby', similarity=sim_pearson))

[(2.9999999999999996, 'the night listener'), (2.8325499182641614, 'lady in water'), (2.3485551731281777, 'just my luck')]


Python 基于协同过滤的推荐 - 影评推荐实例

In [2]:
#此例子不好
#将评分数据从 ratings.dat 中读出到一个 DataFrame 里：
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
#ratings = pd.read_table(r'./ml-1m/ratings.csv', sep=',', header=1, names=rnames)
ratings = pd.read_table(r'./ml-1m/ratings.dat', sep='::', header=None, names=rnames)
ratings[:3]

C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968


In [3]:
#放到一个以 user 为行，movie 为列，rating 为值的表 data 里面。
#（其实将 user 与 movie 的行列关系对调是更加科学的方法，但因为重跑一遍太麻烦了，这里就没改。）
data = ratings.pivot(index='user_id', columns='movie_id', values='rating')
data[:5]

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#稀疏矩阵 - Here's a way to create a sparse dataframe that hopefully should minimize memory and help.
#file = pd.read_csv("./ml-20m/ratings.csv", names=['user_id', 'movie_id', 'rating', 'timestamp'], header=1)
file = pd.read_csv("./ml-1m/ratings.dat", names=['user_id', 'movie_id', 'rating', 'timestamp'], sep='::')

from scipy.sparse import csr_matrix

user_u = list(sorted(file.user_id.unique()))
item_u = list(sorted(file.movie_id.unique()))

row = file.user_id.astype('category', categories=user_u).cat.codes
col = file.movie_id.astype('category', categories=item_u).cat.codes

data = file['rating'].tolist()

sparse_matrix = csr_matrix((data, (row, col)), shape=(len(user_u), len(item_u)))

df = pd.SparseDataFrame([ pd.SparseSeries(sparse_matrix[i].toarray().ravel(), fill_value=0) 
                          for i in np.arange(sparse_matrix.shape[0]) ], 
                        index=user_u, columns=item_u, default_fill_value=0)
df

C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
1,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,4,0,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3,0,0,0,0
10,5,5,0,0,0,0,4,0,0,0,...,0,0,0,0,0,4,0,0,0,0


In [4]:
#接下来要实现推荐的第一步是计算 user 之间的相关系数，DataFrame 对象有一个很亲切的 .corr(method='pearson', min_periods=1) 方法，可以对所有列互相计算相关系数。
#method 默认为皮尔逊相关系数，这个 ok，我们就用这个。
#问题仅在于那个 min_periods 参数，这个参数的作用是设定计算相关系数时的最小样本量，低于此值的一对列将不进行运算。

#min_periods 参数测定
#测定这样一个参数的基本方法为统计在 min_periods 取不同值时，相关系数的标准差大小，越小越好；但同时又要考虑到，我们的样本空间十分稀疏，min_periods 定得太高会导致出来的结果集太小，所以只能选定一个折中的值。 
#这里我们测定评分系统标准差的方法为：在 data 中挑选一对重叠评分最多的用户，用他们之间的相关系数的标准差去对整体标准差做点估计。
#在此前提下对这一对用户在不同样本量下的相关系数进行统计，观察其标准差变化。 
#首先，要找出重叠评分最多的一对用户。我们新建一个以 user 为行列的方阵 foo，然后挨个填充不同用户间重叠评分的个数：
foo = DataFrame(np.empty((len(data.index), len(data.index)), dtype=int), index=data.index, columns=data.index)
for i in foo.index:
    for j in foo.columns:
        foo.ix[i, j] = data.ix[i][data.ix[j].notnull()].dropna().count()

C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  del sys.path[0]


KeyboardInterrupt: 

In [5]:
for i in foo.index:
    foo.ix[i, i] = 0 #先把对角线的值设为 0

In [6]:
ser = Series(np.zeros(len(foo.index)))
for i in foo.index:
    ser[i] = foo[i].max() #计算每行中的最大值

In [7]:
ser.idxmax() #返回 ser 的最大值所在的行号

4169

In [8]:
ser[4169] #取得最大值

577.0

In [9]:
foo[foo==998][4169].dropna() #取得另一个 user_id

Series([], Name: 4169, dtype: float64)

In [10]:
#我们把 424 和 4169 的评分数据单独拿出来，放到一个名为 test 的表里，另外计算了一下这两个用户之间的相关系数为 0.456，还算不错，
#另外通过柱状图了解一下他俩的评分分布情况：
data.ix[4169].corr(data.ix[424])

0.45663851303413283

# 协同过滤
又可分为基于内存的协同过滤和基于模型的协同过滤。
在此教程中，你将实现运用奇异值分解的基于模型协同过滤法和通过计算余弦相似性的基于内存协同过滤法。

In [2]:
#我们将使用MovieLens数据集，这是用来实现和测试推荐引擎最普遍的数据集之一。
#它包含来自于943个用户的10万个电影评价和1682部电影集合。你最好将这个数据集（MoviesLens-100k）解压到你的notebook目录。
header = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('./ml-100k/u.data', sep='\t', names=header)

In [3]:
#先看一眼数据集中的前两行。接下来，让我们统计其中的用户和电影总数。
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_items))

Number of users = 944 | Number of movies = 1682


In [4]:
#你可以使用scikit-learn文库将数据集分为测试和训练两部分。
#Cross_validation.train_test_split 模块根据测试样本的百分比将数据混合并分为两部分，在这里百分比为0.25
from sklearn import cross_validation as cv
train_data, test_data = cv.train_test_split(df, test_size=0.25)

C:\Program Files\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## 基于内存协同过滤法
　　基于内存协同过滤法可以被主要分为两部分：__用户-项目过滤（user-item filtering）__和__项目-项目过滤（ item-item filtering）__。 user-item filtering选取一个特定用户，基于评价相似性找到与该用户相似的其他用户，并推荐那些相似用户所喜欢的项目。相比之下， item-item filtering 先选取一个项目，然后找出也喜欢这个项目的其他用户，并找出这些用户或相似用户也喜欢的其他项目，推荐过程需要项目并输出其他项目。

　　Item-Item Collaborative Filtering: “Users who liked this item also liked …”
User-Item Collaborative Filtering: “Users who are similar to you also liked …”
在这两种情况中，你根据整个数据集创建了一个用户-项目的矩阵。因为已经把数据分成了测试和训练两部分所以你需要创建两个[943 x 1682]矩阵。训练矩阵包含75%的评价，测试矩阵包含25%的矩阵。

　　创建了用户-项目矩阵之后，计算相似性并创建一个相似度矩阵。

Item-Item Collaborative Filtering算法中项目之间的相似度依靠观测所有的已对相同项目评价的用户来测算。

对于User-Item Collaborative Filtering算法，用户之间的相似性依靠观测相同用户已评价的所有项目。

　　推荐系统中通常使用余弦相似性作为距离度量，在n维孔空间中评价被视为向量，基于这些向量之间的夹角来计算相似性。

　　用户a和m可以用下面的公式计算余弦相似性，其中你可以使用用户向量 $u_k$ 和 $u_a$ 之间的点积然后除以这两个向量欧式长度之乘。
$$\large{s_u^{cos}(u_k , u_a)=\frac{u_k \cdot u_a}{||u_k|| \ ||u_a||}=\frac{\sum{x_{k,m} \  x_{a,m}}}{\sqrt{\sum{x_{k,m}^2}\sum{x_{a,m}^2}}}}$$

　　而计算项目m和b之间的相似度可以用下面的公式：
$$\large{s_u^{cos}(i_m , i_b)=\frac{i_m \cdot i_b}{||i_m|| \ ||i_b||}=\frac{\sum{x_{a,m} \  x_{a,b}}}{\sqrt{\sum{x_{a,m}^2}\sum{x_{a,b}^2}}}}$$

首先创建user-item矩阵，因此你需要创建两个矩阵为测试和训练数据集。

In [6]:
#Create two user-item matrices, one for training and another for testing
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3] 

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

In [7]:
#你可以使用 sklearn 的pairwise_distances函数来计算余弦相似性。注意，因为评价都为正值输出取值应为0到1.
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

下一步是作出预测。既然构造了相似度矩阵user_similarity和item_similarity，因此你可以运用下面的公式为user-based CF做一个预测：
$$\large{\hat x_{k,m}=\overline x_k + \frac{\sum_{u_a}^{}{sim_u (u_k , u_a)(x_{a,m}-\overline x_{u_a})}}{\sum_{u_a}^{}{\mid sim_u (u_k , u_a)\mid}}}$$
当为item-based CF做一个推荐时候，你不要纠正用户的平均评价，因为用户本身用查询来做预测。
$$\large{\hat x_{k,m}=\frac{\sum_{i_b}^{}{sim_i (i_m , i_b)(x_{k,b})}}{\sum_{i_b}^{}{\mid sim_i (i_m , i_b) \mid}}}$$

In [8]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])    
    return pred

item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

评估　

有许多的评价指标，但是用于评估预测精度最流行的指标之一是Root Mean Squared Error (RMSE)。
$$RMSE=\sqrt{\frac{1}{N} \sum{(x_i - \hat x_i)^2}}$$


 

你可以用sklearn中的 mean_square_error (MSE)函数，RMSE只是MSE其中的一个平方根。 

因为你仅仅想考虑在这个测试数据集中的预测评价，你可以用prediction[ground_truth.nonzero()]过滤测试矩阵中所有其他的元素。

In [10]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 3.124317308976175
Item-based CF RMSE: 3.452148420781322


## Model-based Collaborative Filtering

基于模型的协同过滤是基于已收到更大的曝光，主要是作为潜变量分解和降维无监督的学习方法矩阵分解（MF）
Model-based Collaborative Filtering基于已收到很多曝光的矩阵因式分解 （MF），主要是作为潜在变量分解和降维无监督学习方法。矩阵因式分解因其能比Memory-based CF更好解决可扩展性和稀疏问题而被广泛用于推荐系统。MF的目标是学习用户潜在的喜好和从已知评分的项目的潜在属性（学习描述评分特征的特点），以随后通过用户和项目的潜在特征点积预测未知的评分。

当你有一个多维度稀疏矩阵，通过矩阵因式分解你能够将用户-项目矩阵（user-item matrix）重构成低评分结构（low-rank structure），并且你能够通过两个低评分（low-rank）矩阵相乘得出这个矩阵，其中矩阵的行包含潜在向量。

通过低评价矩阵乘积尽可能调整这个矩阵近似原始矩阵，以填充原始矩阵中缺失的项。

现在开始计算MovieLens 数据集的稀疏等级：

In [12]:
sparsity = round(1.0 - len(df) / float(n_users * n_items), 3)
print('The sparsity level of MovieLens100K is ' +  str(sparsity*100) + '%')

The sparsity level of MovieLens100K is 93.7%


In [30]:
#例子不好
#!pip install texttable
#基于Python2.7   数据集为MovieLens 100k数据集
 
# -*-coding=utf-8 -*-
import sys
import math
from texttable import Texttable
 
#算法流程：
#1、建立电影-用户的倒排表，表示电影被那些用户看过
#2、从目标用户看过的每一步电影开始遍历
#3、寻找和目标用户有共同看过电影交集的所有用户作为初始邻居
#4、对初始邻居里边的每一个邻居，计算和目标用户的余弦夹角相似度
#5、建立相似度到用户的倒排表，相似度从高到低排列
#6、根据相似度找出最近k邻居
#7、从最近k邻看过的所有的电影开始遍历
#  这里的推荐思路是：对于最近k邻居看过的所有电影中的某一电影m
#  如果m仅仅被一个邻居看过，那么目标用户对此电影的的兴趣度就是目标用户和这个邻居的相似度
#  如果m被多个邻居看过，那么目标用户对此电影的相似度为目标用户与这些邻居相似度之和
#8、建立目标用户兴趣度-电影id的倒排表
#9、根据兴趣度由大到小进行推荐 
 
#计算余弦距离
#listUser2Score[2] = [(1,5), (4,2)].... 表示用户2对电影1的评分是5，对电影4的评分是2
#dist = getCosDist(listUser2Score[userId], listUser2Score[neighbor])
def getCosDist(user1, user2):
    sum_x = 0.0
    sum_y = 0.0
    sum_xy = 0.0
    for key1 in user1:
        for key2 in user2:
# key1[0]表示电影id，key1[1]表示对电影的评分
# 如果是两个用户共同评价的一部电影
            if key1[0] == key2[0]:
                sum_x += key1[1] * key1[1]
                sum_y += key2[1] * key2[1]
                sum_xy += key1[1] * key2[1]
    if sum_xy == 0.0:
        return 0
    demo = math.sqrt(sum_x * sum_y)
    return sum_xy / demo
 
#读取文件，读取以行为单位，每一行是列表里的一个元素
def readFile(filename):
    contents = []
    f = open(filename, "r", encoding='utf-8')
    contents = f.readlines()
    f.close()
    return contents
 
#数据格式化为二维数组
def getRatingInfo(ratings):
    rates = []
    for line in ratings:
        rate = line.split("\t")
#  u.data
#  user id | item id | rating | timestamp.
        rates.append([int(rate[0]), int(rate[1]), int(rate[2])])
    return rates
 
#生成用户评分数据结构
def getUserScoreDataStructure(rates):    
#listUser2Score[2]=[(1,5), (4,2)].... 表示用户2对电影1的评分是5，对电影4的评分是2
    listUser2Score = {}
#dictItem2Users{}, key=item id, value=user id list
    dictItem2Users = {}
    for k in rates:
#  u.data
#  user id | item id | rating | timestamp.
        user_rank = (k[1], k[2])
        if k[0] in listUser2Score:
            listUser2Score[k[0]].append(user_rank)
        else:
            listUser2Score[k[0]] = [user_rank]
 
        if k[1] in dictItem2Users:
            dictItem2Users[k[1]].append(k[0])
        else:
            dictItem2Users[k[1]] = [k[0]]
    return listUser2Score, dictItem2Users
 
#计算与目标用户
def getNearestNeighbor(userId, listUser2Score, dictItem2Users):
    neighbors = []
#listUser2Score[2]=[(1,5),(4,2)].... 表示用户2对电影1的评分是5，对电影4的评分是2
# 对于目标用户userId的每一个评价过的项目item
    for item in listUser2Score[userId]:
#dictItem2Users{},key=item id,value=user id list
#item[0]表示电影id，item[1]表示电影评分
#dictItem2Users[item[0]]=dictItem2Users[电影id]=value=评价过这个电影的用户列表
#从评价过这个电影的用户列表里，计算目标用户和这个列表里边所有用户的相似度
        for neighbor in dictItem2Users[item[0]]:
#如果这个邻居不是目标用户并且这个邻居还没有被加入邻居集就加进来
            if neighbor != userId and neighbor not in neighbors:
                neighbors.append(neighbor)
    neighbors_dist = []
#里边存储的是[相似度，邻居id]
    for neighbor in neighbors:
#listUser2Score[2]=[(1,5),(4,2)].... 表示用户2对电影1的评分是5，对电影4的评分是2
        dist = getCosDist(listUser2Score[userId], listUser2Score[neighbor])
        neighbors_dist.append([dist, neighbor])
#  按照相似度倒排，相似度从道到低
    neighbors_dist.sort(reverse = True)
    return neighbors_dist
 
#使用UserFC进行推荐，输入：文件名,用户ID,邻居数量
def recommendByUserFC(filename, userId, k = 5):
#读取文件
    contents = readFile(filename)
#文件格式数据转化为二维数组
    rates = getRatingInfo(contents)
#格式化成字典数据
    listUser2Score, dictItem2Users = getUserScoreDataStructure(rates)
#找邻居
#找出最相似的前五个邻居
    neighborsTopK = getNearestNeighbor(userId, listUser2Score, dictItem2Users)[:5]
#neighborsTopK存储了相似度和邻居id的倒排表
#所以neighbor[1]表示邻居id，neighbor[0]表示相似度
#这里的推荐思路是：对于最近k邻居看过的所有电影中的某一电影m
#如果m仅仅被一个邻居看过，那么目标用户对此电影的的兴趣度就是目标用户和这个邻居的相似度
#如果m被多个邻居看过，那么目标用户对此电影的相似度为目标用户与这些邻居相似度之和
#建立推荐字典
    recommand_dict = {}
    for neighbor in neighborsTopK:
        neighbor_user_id = neighbor[1]
#找出这个邻居看过的所有电影信息 
        movies = listUser2Score[neighbor_user_id]
        for movie in movies:
            if movie[0] not in recommand_dict:
                recommand_dict[movie[0]] = neighbor[0]
            else:
                recommand_dict[movie[0]] += neighbor[0]
#建立推荐列表
    recommand_list = []
    for key in recommand_dict:
#建立目标用户兴趣度-电影id的倒排表    
        recommand_list.append([recommand_dict[key], key])
    recommand_list.sort(reverse = True)
#listUser2Score列表里边的元素是电影id到电影评分的键值对
#所以这里的k[0]表示电影id，k[1]表示电影评分
    user_movies = [k[0] for k in listUser2Score[userId]]
#recommand_list存储的是目标用户兴趣度到电影id的倒排表
#所以这里的的k[1]表示的是电影id，k[0]表示的是兴趣度
    #return [k[1] for k in recommand_list], user_movies, dictItem2Users, neighbors
    return [k[1] for k in recommand_list], user_movies, dictItem2Users, neighborsTopK
 
#获取电影的列表
def getMovieList(filename):
    contents = readFile(filename)
    movies_info = {}
    for movie in contents:
        single_info = movie.split("|")
        movies_info[int(single_info[0])] = single_info[1:]
    return movies_info
 
#从这里开始运行    
if __name__ == '__main__':
    #reload(sys)
    #sys.setdefaultencoding('utf-8')
#获取所有电影的列表,所有电影id到电影名字的键值对
    dictMovieId2Info = getMovieList("./ml-100k/u.item")
    listRecommendMovieId, user_movie, items_movie, neighbors = recommendByUserFC("./ml-100k/u.data", 50, 80)
    neighbors_id=[ i[1] for i in neighbors]
    table = Texttable()
    table.set_deco(Texttable.HEADER)
    table.set_cols_dtype(['t', 't', 't'])
    table.set_cols_align(["l", "l", "l"])
    rows=[]
    rows.append([u"movie name",u"release time", u"from userid"])
#打印推荐列表的前20项数据，listRecommendMovieId里边存储的仅仅是id
    for movie_id in listRecommendMovieId[:20]:
        from_user=[]
        for user_id in items_movie[movie_id]:
            if user_id in neighbors_id:
                from_user.append(user_id)
#dictMovieId2Info[movie_id][0]表示电影名 dictMovieId2Info[movie_id][1]表示时间
        rows.append([dictMovieId2Info[movie_id][0],dictMovieId2Info[movie_id][1],from_user])
    table.add_rows(rows)
    print(table.draw())

               movie name                   release time       from userid     
When Harry Met Sally... (1989)              01-Jan-1989    [942, 904, 901, 934]
Dave (1993)                                 01-Jan-1993    [904, 934, 901]     
African Queen, The (1951)                   01-Jan-1951    [901, 934, 942]     
Grease (1978)                               01-Jan-1978    [904, 901, 934]     
Butch Cassidy and the Sundance Kid (1969)   01-Jan-1969    [934, 901, 942]     
E.T. the Extra-Terrestrial (1982)           01-Jan-1982    [934, 942, 901]     
Air Force One (1997)                        01-Jan-1997    [909, 904, 942]     
Sense and Sensibility (1995)                01-Jan-1995    [901, 909, 904]     
Jerry Maguire (1996)                        13-Dec-1996    [934, 904, 901]     
Jaws (1975)                                 01-Jan-1975    [901, 942, 934]     
Indiana Jones and the Last Crusade (1989)   01-Jan-1989    [934, 901, 942]     
Return of the Jedi (1983)               